# snscrape tweets

## Imports

In [ ]:
import snscrape.modules.twitter as sntwitter

import pandas as pd

import time

import datetime

from sqlalchemy import create_engine

## Scrape tweets

In [119]:
tweets_list = []
hashtags = "bitcoin OR btc"
since_date = datetime.datetime(2019,10,1)
until_date = datetime.datetime(2019,10,2)
end_date = datetime.datetime(2021,11,1)
first_run = True
df = pd.DataFrame()
max_tweets_per_day = 10000
max_tweets_total = 4000000

In [120]:
def create_df(tweets_list):
    df = pd.DataFrame(tweets_list, columns=[
                        "url",
                        "date",
                        "content",
                        "renderedContent",
                        "tweet_id",
                        "replyCount",
                        "retweetCount",
                        "likeCount",
                        "quoteCount",
                        "conversationId",
                        "lang",
                        "quotedTweet",
                        "inReplyToTweetId",
                        "coordinates",
                        "place",
                        "hashtags",
                        "user_id",
                        "user_username",
                        "user_description",
                        "user_verified",
                        "user_created",
                        "user_followersCount",
                        "user_friendsCount",
                        "user_statusesCount",
                        "user_favouritesCount",
                        "user_location"])
    return df

In [121]:
while(since_date != end_date and df.shape[0] < max_tweets_total ):
    
    q = '{} lang:en since:{} until:{} -filter:replies'.format(hashtags,since_date,until_date)
    
    try:
        for i,tweet in enumerate(sntwitter.TwitterSearchScraper(q).get_items()):
            if i > max_tweets_per_day:
                break
            tweets_list.append([
                        tweet.url,
                        tweet.date,
                        tweet.content,
                        tweet.renderedContent,
                        tweet.id,
                        tweet.replyCount,
                        tweet.retweetCount,
                        tweet.likeCount,
                        tweet.quoteCount,
                        tweet.conversationId,
                        tweet.lang,
                        tweet.quotedTweet,
                        tweet.inReplyToTweetId,
                        tweet.coordinates,
                        tweet.place,
                        tweet.hashtags,
                        tweet.user.id,
                        tweet.user.username,
                        tweet.user.description,
                        tweet.user.verified,
                        tweet.user.created,
                        tweet.user.followersCount,
                        tweet.user.friendsCount,
                        tweet.user.statusesCount,
                        tweet.user.favouritesCount,
                        tweet.user.location
                        ])
    except Exception as e:
        print(e)
            
    if first_run == True and len(tweets_list) != 0:
        df = create_df(tweets_list)
        first_run = False
        
    elif first_run == False:
        df = df.append(create_df(tweets_list))
        
    print("Since: ",since_date, " until: ", until_date, " new tweets: ",len(tweets_list), " total tweets: ", df.shape[0] )
    since_date += datetime.timedelta(days=1)
    until_date += datetime.timedelta(days=1)
    tweets_list = []

Since:  2019-10-01 00:00:00  until:  2019-10-02 00:00:00  new tweets:  579  total tweets:  579
Since:  2019-10-02 00:00:00  until:  2019-10-03 00:00:00  new tweets:  545  total tweets:  1124
Since:  2019-10-03 00:00:00  until:  2019-10-04 00:00:00  new tweets:  537  total tweets:  1661
Since:  2019-10-04 00:00:00  until:  2019-10-05 00:00:00  new tweets:  492  total tweets:  2153
Since:  2019-10-05 00:00:00  until:  2019-10-06 00:00:00  new tweets:  509  total tweets:  2662
Since:  2019-10-06 00:00:00  until:  2019-10-07 00:00:00  new tweets:  490  total tweets:  3152
Since:  2019-10-07 00:00:00  until:  2019-10-08 00:00:00  new tweets:  465  total tweets:  3617
Since:  2019-10-08 00:00:00  until:  2019-10-09 00:00:00  new tweets:  461  total tweets:  4078
Since:  2019-10-09 00:00:00  until:  2019-10-10 00:00:00  new tweets:  483  total tweets:  4561
Since:  2019-10-10 00:00:00  until:  2019-10-11 00:00:00  new tweets:  441  total tweets:  5002
Since:  2019-10-11 00:00:00  until:  2019

## Create df

In [127]:
df_tweet = df
df_tweet["hashtags"] = df_tweet.hashtags.apply(lambda x: str(x))
df_tweet["quotedTweet"] = df_tweet.quotedTweet.apply(lambda x: str(x))
df_tweet["coordinates"] = df_tweet.coordinates.apply(lambda x: str(x))
df_tweet["place"] = df_tweet.place.apply(lambda x: str(x))
df_tweet["user_created"] = df_tweet.user_created.apply(lambda x: str(x))
df_tweet["date"] = df_tweet.date.apply(lambda x: str(x))

In [97]:
df_tweet.hashtags = df_tweet.hashtags.apply(lambda x: str(x).replace("[","").replace("]","").replace(",","").replace("'",""))

In [135]:
df_tweet = df_tweet.reset_index(drop = True)

## Save df to excel

In [ ]:
df_tweet.to_csv('Tweets_unedited.csv',index=False)

## Save df in MySQL

In [137]:
from sqlalchemy import create_engine

In [138]:
en = create_engine('mysql+mysqlconnector://duinmat:$Mo/qdGSnX$MEi@oege.ie.hva.nl/zduinmat')

In [139]:
df_tweet.to_sql("Tweets_unedited",con = en,if_exists='replace',chunksize=10000)